Import Libraries

In [1]:
import sqlite3
import requests
from tqdm import tqdm

from flask import Flask, request
import json 
import numpy as np
import pandas as pd

# Introduction
In most cases when we are running a business, there are a lot of data stakeholder outside our company. The problem is that we need to provide the access in a way that they will not break our security rules or concerns. One way to solve that is by creating an API for the database. In this project, we will introduce you on how python is used for data transaction management using Flask API. 

**Usecase**: Bikeshare App\
Have you ever rent a bike for faster mobility in town? In the past few years, this business once become a phenomenom. In Indonesia, there are lots of similar services, for example, the Jakarta government's "GOWES" bike sharing service that launcehd in July 2020. 

For the user perspective, the general journey is denoted as follows:
- User scan the bike located at some bike station, sending the data to database as the intent of "start riding"
- Once user has reached its destination station, he/she put back the bike, sending the data again to the database as the intent of "finished riding"

For each activity, there are data transactions between user and database. And how do you think each user's phone communicate with the server for storing and receiving the data? Using API ofcourse! 

We will later create a simplified version of the API service which handles data transactions and analysis. 


**Goals**: Make an API service to connect 3rd party user with data using HTTP request

**End Product**: A Flask API which capable of doing: 
- Input new data to database
- Read specific data from database
- Get specific insight from data analysis process (ie: best performing stations)

**Scoring Metrics**: 

1. 1 point - Created Flask App

-  create app.py file to make flask app
-  create Flask app to execute all of the endpoint you have made

2. 2 points - Created functionality to read or get specific data from the database

- create query to read data from database
- create function to execute read specific information into table from database

3. 4 points - Created functionality to input new data into each table for the databases

- create query to insert new data into stations and trips table
- create function to execute input data into stations and trips table

4. 3 points - Created static endpoints which return analytical result (must be different from point 2,3)

- create query to make analytical resultfrom the data
- create static endpoint to analyze the data from database, for example average trip durations
    
5. 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)

- create query to make analytical resultfrom the data
- create dinamic endpoint to analyze the data from database, for example average trip durations for each bike_id

6. 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)

- create input data for refering into query for post endpoint
- make query and aggregation function to implement the input

**Tools**: 
- **Python** with **Jupyter Notebook**, installed with **required libraries**
- **Visual Studio Code (VSCode)**: Recommended for writing application scripts
- **TablePlus**: Recommended for easy database access and exploration
- Postman: Optional for easy API testing


# About the Data

The data that we will be using in this project is **[Austin Bike Share]()** dataset which contains information on bike trip start location, stop location, duration, type of bike share user in the city of Austin, Texas. Bike station location data is also provided. 

All the information is stored in a database called **austin_bikeshare.db**. However, we also provides the non existing data in csv files. These data then will be imported into the database using the API

Lists of files:
- **austin_bikeshare.db**: The database, contains `trips` and `stations` table
- **data/austin_bikeshare_stations.csv**: contains all the stations information which is not yet available in the database table
- **data/austin_bikeshare_trips_2021.csv**: contains all 2021 data which is not yet available in the database table

In [2]:
# Reading the csv data
trips = pd.read_csv('data/austin_bikeshare_trips_2021.csv')
stations = pd.read_csv('data/austin_bikeshare_stations.csv')

## Taking a look for trips data
`trips` table in database, or `austin_bikeshare_trips.csv` in original files. It roughly consisted of 1.3 million rows

In [3]:
trips.head(2)

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16


**Data Descriptions**:
- `bikeid`: integer id of bike
- `checkout_time`: HH:MM:SS, see start time for date stamp
- `duration_minutes`: int minutes of trip duration
- `endstationid`: integer id of end station
- `endstationname`: string of end station name
- `month`: month, integer
- `startstationid`: integer id of start station
- `startstationname`: string of start station name
- `start_time`: YYYY-MM-DD HH:MM:SS
- `subscriber_type`: membership typ e.g. walk up, annual, other bike share, etc
- `trip_id`: unique trip id int
- `year`: year of trip, int

## Taking a look for stations data
`stations` table in database, or `austin_bikeshare_stations.csv` in original files 

In [4]:
stations.tail(2)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,NaN,32671.0,undetermined_parking,19.0,solar,55.0,5.0,NaN,9,2021-01-04T12:00:00Z
97,3797,21st & University,active,2007 University Ave.,NaN,32674.0,undetermined_parking,19.0,solar,55.0,5.0,ROW might belong to UT,9,2021-01-04T12:00:00Z


**stations table (or austin_bikeshare_trips.csv)**:
- `station_id`: integer id of station
- `name`: string of station name
- `status`: string of station status (active, closed, moved, ACL-only)
- `address`: string of station address
- `alternate_name`: string of station alternative name
- `city_asset_number`: integer of station's asset number
- `property_type`: string of station's property type
- `number_of_docks`: integer of number of available bike docks
- `power_type`: string of station's power source type
- `footprint_length`: float of station' blueprint length (size of the station). Probably in meters
- `footprint_width`: float of station' blueprint width (size of the station). Probably in meters
- `notes`: string of additional notes
- `council_district`: integer of stations's council district
- `modified_date`: date of last modified information regarding the station

# Database

The first important task is to make sure we can securely make connections to the database. In this scenario, we will connect to sqlite database, `austin_bikeshare.db` via python. In this part, we will re-visit on how to work with databases, started with making a connection, get some data, and insert data into it. 

You can directly connect and view the database using TablePlus, or run the following code to create the connection

In [5]:
# Define a function to create connection for reusability purpose
def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection

# Make a connection
conn = make_connection()

In [6]:
conn

## POST: Insert data into database

Generally, POST method will utilize the data sent by user for specific purporse, for example: 

- Insert new data into the database. 
- Operate the data into some function

We will learn how to insert data into a specific table in our database. Please refer to the following code to create and run the query for the given task

In [7]:
stations.iloc[1] # series

station_id                           2500
name                      Republic Square
status                             closed
address                  425 W 4th Street
alternate_name                        NaN
city_asset_number                     NaN
property_type                         NaN
number_of_docks                       NaN
power_type                            NaN
footprint_length                      NaN
footprint_width                       NaN
notes                                 NaN
council_district                        9
modified_date        2021-01-04T12:00:00Z
Name: 1, dtype: object

In [8]:
# Get the data values
data = tuple(stations.iloc[1].fillna('').values) # cast series to tuple to fit insert query format
data

# query doesn't recognized NaN, change to empty string instead

(2500,
 'Republic Square',
 'closed',
 '425 W 4th Street',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 9,
 '2021-01-04T12:00:00Z')

In [9]:
# format
# insert_query = "INSERT INTO <TABLE> VALUES (VALUE1, VALUE2, VALUE3, ..., VALUEN)"

# Make the query
query = f"""
INSERT INTO stations
VALUES {data}
"""
# dynamic string format from python using {var}

In [10]:
# See the actual query looks like
print(query)


INSERT INTO stations
VALUES (2500, 'Republic Square', 'closed', '425 W 4th Street', '', '', '', '', '', '', '', '', 9, '2021-01-04T12:00:00Z')



In [11]:
# Execute the query 
conn.execute(query)

In [12]:
# Commit changes to database
conn.commit()

Once the code above succesful, it's recomended to wrap it into a function so that we can reuse it in the future. Complete the following codes to make the function:

In [13]:
def insert_into_stations(data, conn):
    query = f"""INSERT INTO stations values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [14]:
# Example use of the function 
conn = make_connection()
data = tuple(stations.iloc[6].fillna('').values) # Randomly select a data
result = insert_into_stations(data, conn)

In [15]:
result

'OK'

**TASK: Create a Function to insert data for `trips` table**

In [21]:
# Your code here
def insert_into_trips(data, conn):
    query = f"""INSERT INTO trips values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

## GET: Read specific data

Generally, GET method will ask for specific the data in the database, alongside with additional information we might send. For example: 
- Get number of unique user
- Get full trips information for specific user id
- Get average trips duration and length for specific user id

In this example, we will implement the most basic GET method, that is to get Station information in the table based on specific Station ID

In [22]:
# Get Specific Station ID  Query
station_id = 2500
query_station_id = f"""
SELECT * FROM stations
WHERE station_id = {station_id}
"""

# Get all station ID Query
query_station_all = "SELECT * FROM stations"

In [23]:
print(query_station_id)


SELECT * FROM stations
WHERE station_id = 2500



In [24]:
# Run the query and get the result
result = pd.read_sql_query(query_station_id, conn)

In [25]:
result

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


Just like the previous section, create a function to read specific station

In [26]:
def get_station_id(station_id, conn):
    query = f"""SELECT * FROM stations WHERE station_id = {station_id}"""
    result = pd.read_sql_query(query, conn)
    return result 

def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result

**TASK: Create a Function to read data from `trips` table**

In [27]:
# Your code here
def get_trip_id(trip_id, conn):
    query = f"""SELECT * FROM stations WHERE trip_id = {trip_id}"""
    result = pd.read_sql_query(query, conn)
    return result 
    
def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

def get_avg_dur_trips(conn):
    query = f"""SELECT AVG(duration_minutes) AS AvgDur FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

# Flask App

Flask is a micro-framework for python. Generally we can build any application out of it. To start with, let's make our first empy flask app. 
Creata a new file `app.py`, then copy-paste the following codes into it and save it. 

*Notes: it's recomended to use vscode as editor since it support vast linting supports, including python which we find really helpful in coding*

```python
from flask import Flask, request
app = Flask(__name__) 

if __name__ == '__main__':
    app.run(debug=True, port=5000)
```

To run the app, you can open your terminal, go to the specific folder and run `python app.py` using your designated virtual environment

## Routes and Endpoints

### Implement endpoints

If you succesfully run the previous app and access the localhost:5000 , you might get an 404 not found error. This is because we **have not** define yet what will happen if people accessing our root path of the app (localhost:5000/)

Add the following example of route or endpoint into your `app.py` just before the `if __name__ === '__main__':` line, and reload the page to see if it works

```python 
@app.route('/')
def home():
    return 'Hello World'
```

Above endpoints will runs the `home` function anytime user access the `/` page.

Since we are going to handle all the requests through this app, the very next step is to implement our functionalites. Let's start with implementing the functionality to read all station data

we will add the `get_all_stations()` functions into our app, create a `@app.route('/stations/')` endpoint and call the function in it. The code should looks like:

```python 
@app.route('/stations/')
def route_all_stations():
    conn = make_connection()
    stations = get_all_stations(conn)
    return stations.to_json()
    
def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result
```

However, adding just above codes is not enough. If you see, there is a dependency inside the function, which is `make_connection()` function. Other than that, we will be using all the required libraries. 

Hence, we can complete the code by : 
- import the required libraries at the top of the `app.py` file
- write the `make_connection()` function before any routes declarations

Once it's completed, you should see no errors in the vscode screen, and your endpoint should work just fine and returns all the stations. It's now your task to impelement the `get_all_trips` into the app 

**TASK: Implement `get_all_trips()` method into the app** \
Place it under `@app.route('/trips/')` endpoint

In [28]:
# your code
@app.route('/trips/')
def route_all_trips():
    conn = make_connection()
    trips = get_all_trips(conn)
    return trips.to_json()
    
def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

NameError: name 'app' is not defined

### Access Endpoints

To access our endpoints using python, simply use `requests` library followed by the request method (post, get, put, delete, etc) the passed in the url or data 

In [34]:
url = 'http://127.0.0.1:5000/stations/'
res = requests.get(url)

In [30]:
res

<Response [200]>

The response sent by the server was not only contains bare data. It was packed as HTTP response, meaning that we need to unpack the response first  in order to get the data. we can use `res.json()` to obtain it, then use pandas to transform it into dataframe for readibility or future analysis purposes. 

In [31]:
res.text

'{"station_id":{"0":2500,"1":2546},"name":{"0":"Republic Square","1":"ACC - West & 12th Street"},"status":{"0":"closed","1":"closed"},"address":{"0":"425 W 4th Street","1":"1231 West Ave."},"alternate_name":{"0":"","1":""},"city_asset_number":{"0":"","1":""},"property_type":{"0":"","1":""},"number_of_docks":{"0":"","1":""},"power_type":{"0":"","1":""},"footprint_length":{"0":"","1":""},"footprint_width":{"0":"","1":""},"notes":{"0":"","1":""},"council_district":{"0":9,"1":9},"modified_date":{"0":"2021-01-04T12:00:00Z","1":"2021-01-04T12:00:00Z"}}'

In [32]:
res.json()

{'station_id': {'0': 2500, '1': 2546},
 'name': {'0': 'Republic Square', '1': 'ACC - West & 12th Street'},
 'status': {'0': 'closed', '1': 'closed'},
 'address': {'0': '425 W 4th Street', '1': '1231 West Ave.'},
 'alternate_name': {'0': '', '1': ''},
 'city_asset_number': {'0': '', '1': ''},
 'property_type': {'0': '', '1': ''},
 'number_of_docks': {'0': '', '1': ''},
 'power_type': {'0': '', '1': ''},
 'footprint_length': {'0': '', '1': ''},
 'footprint_width': {'0': '', '1': ''},
 'notes': {'0': '', '1': ''},
 'council_district': {'0': 9, '1': 9},
 'modified_date': {'0': '2021-01-04T12:00:00Z', '1': '2021-01-04T12:00:00Z'}}

In [33]:
pd.DataFrame(res.json())#.head()

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z
1,2546,ACC - West & 12th Street,closed,1231 West Ave.,,,,,,,,,9,2021-01-04T12:00:00Z


In [35]:
url_trips = 'http://127.0.0.1:5000/trips/'
res_trips = requests.get(url_trips)

In [36]:
pd.DataFrame(res_trips.json()).head()

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00 UTC,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19


## Static and Dynamic Endpoints

On previous part we already made several endpoints which control how our server will react whenever users access it. 
If we take a look at the endpoints, it's all static : 
- `@app.route('/')`
- `@app.route('/home/')`
- `@app.route('/stations/')`
- `@app.route('/trips/')`


What if, instead of getting all the stations information, we only wanted to read a specific station information of station_id 3464 ? \
should we make an excact endpoint of that like `@app.route('stations/3464')`?. But what about the others? are we going to make a bunch of endpoints for each specific id like: 
- `@app.route('/stations/3464')`
- `@app.route('/stations/2500')`
- `@app.route('/stations/2541')`
- and so on...

Of course we shouldn't. One way to overcome the problem is by delcaring a **dynamic endpoints**. Basically, it's an endpoint which allows user to insert a variable values in it. The final looks of the dynamic endpoint version of above problem is: 
- `@app.route('stations/<station_id>')`

From above endpoint, the \<station_id\> is the variable, and we need to make sure that it influence how our following function acts. The whole dynamic endpoints for getting specific station by its id will looks like: 

```python 
@app.route('/stations/<station_id>')
def route_stations_id(station_id):
    conn = make_connection()
    station = get_station_id(station_id, conn)
    return station.to_json()
```


Before accesing the endpoint, make sure that `make_connection()` and `get_station_id()` functions are included inside `app.py` script so that it won't raise an error. Now for the final step, we can try to access the dynamic endpoints by changing the `station_id` in the following codes

In [44]:
station_id = 2500
url = f"http://127.0.0.1:5000/stations/{station_id}"

In [41]:
url

'http://127.0.0.1:5000/stations/2500'

In [42]:
response = requests.get(url)
pd.DataFrame(response.json())

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


**TASK: Implement dynamic endpoints to read specific trip by its trip_id!** \
Place it under `@app.route('/trips/<trip_id>')` endpoint

`@app.route('/trips/<trip_id>')
def route_trips_id(trip_id):
    conn = make_connection()
    trip = get_trip_id(trip_id, conn)
    return trip.to_json()`

In [51]:
trip_id = 9900289692
url = f"http://127.0.0.1:5000/trips/{trip_id}"

In [52]:
url

'http://127.0.0.1:5000/trips/9900289692'

In [53]:
response = requests.get(url)
pd.DataFrame(response.json())

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39


## Handling JSON data as input

Sometimes, in order to make something happens in our API, we need the user to send us the data. In this case, we need to handle how we can get the data (which mostly sent as json format) and utilize it inside our endpoint functions. 

In order to achieve that, we will be using flask's `request` classes.


```python
@app.route('/json') 
def json_example():
    
    req = request.get_json(force=True) # Parse the incoming json data as Dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')
```

In [54]:
data = {
    "name" : "Andi", 
    "age" : 24,
    "address" : "my adress"
}

url = "http://127.0.0.1:5000/json"

In [58]:
res = requests.get(url, json=data)

In [59]:
res

<Response [200]>

In [60]:
res.text

'Hello Andi, your age is 24, and your address in my adress\n            '

In [55]:
res = requests.post(url, json=data) # if we try using post method

In [56]:
res # error

<Response [405]>

In [57]:
res.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>405 Method Not Allowed</title>\n<h1>Method Not Allowed</h1>\n<p>The method is not allowed for the requested URL.</p>\n'

One endpoint can be accessed with several method. But by default an endpoint has GET method. So, if we want to use POST method we add `method=['POST']` as parameter in `@app.route`


```python
@app.route('/json', methods=['POST']) 
def json_example():
    
    req = request.get_json(force=True) # Parse the incoming json data as Dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')
```

In [63]:
data = {
    "name" : "Andi", 
    "age" : 24,
    "address" : "my adress"
}

url = "http://127.0.0.1:5000/json"

In [66]:
res = requests.post(url, json=data)

In [67]:
res

<Response [200]>

In [68]:
res.text

'Hello Andi, your age is 24, and your address in my adress\n            '

Now that we already know how to handle json input, we can try to implement and endpoint in which we can insert new data into `stations` table. 

```python

@app.route('/stations/add', methods=['POST']) 
def route_add_station():
    # parse and transform incoming data into a tuple as we need 
    data = pd.Series(eval(request.get_json(force=True)))
    data = tuple(data.fillna('').values)
    
    conn = make_connection()
    result = insert_into_stations(data, conn)
    return result


```

In [71]:
stations.iloc[20].fillna('').to_json() # can use this but sometimes not working well with our API

'{"station_id":2502,"name":"Barton Springs & Riverside","status":"closed","address":"400 Barton Springs Rd.","alternate_name":"","city_asset_number":"","property_type":" ","number_of_docks":"","power_type":" ","footprint_length":"","footprint_width":"","notes":"moved to Pedernales\\/2nd. Original city ID: 16729","council_district":9,"modified_date":"2021-01-04T12:00:00Z"}'

In [74]:
# so we will cast to dictionary instead
data = stations.iloc[15].fillna('').to_dict()

In [76]:
# and we'll try to transform the data to JSON by using json.dumps
json.dumps(data) # error object of tye int64 is not JSON serializable the datatype in our dataframe is not what casually there in the internet

'{"station_id": 1004, "name": "Red River & LBJ Library", "status": "closed", "address": "2322 Red River Street", "alternate_name": "", "city_asset_number": "", "property_type": "", "number_of_docks": "", "power_type": "", "footprint_length": "", "footprint_width": "", "notes": "", "council_district": 1, "modified_date": "2021-01-04T12:00:00Z"}'

In [77]:
# so we need encoding to change numpy data type to simple data type
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [80]:
data = stations.iloc[20].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)

In [81]:
url = "http://127.0.0.1:5000/stations/add"
res = requests.post(url, json=data_json)

In [82]:
res

<Response [200]>

In [83]:
res.text

'OK'

In [84]:
# Syntax for iteratively insert stations (from stations_csv) into stations table

for idx, data in tqdm(stations.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/stations/add"
    res = requests.post(url, json=data_json)

98it [00:01, 95.87it/s]


**TASK: Using the API, insert all the trips data (in csv) into the database by running the following codes**\
It might run differently on each devices, so you might get some rest while waiting for it to complete

In [93]:
for idx, data in tqdm(trips.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/trips/add"
    res = requests.post(url, json=data_json)

17982it [06:46, 44.23it/s]


In [94]:
res

<Response [200]>

# Make your own analytic endpoints

Based on what we've learned before, create your own endpoints which returns analytical result such as contigency tables, aggregation tables, or even just a values.

*ps: if the return is dataframe/series, don't forget to change it into json with `.to_json()` method*


- [x] 1 point - Created Flask App
- [x] 4 points - Created functionality to input new data into each table for the databases
- [x] 2 poitns - Created functionality to read or get specific data from the database
- [ ] 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
- [ ] 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
- [ ] 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)


## Create Static Endpoint(s)

You can use the following cell to try-out your function before implementing it as an endpoint

In [ ]:
# Your Code Here 
@app.route('/trips/average_duration') # -> bisa semua riding dikalkulasi atau agregasi tiap station dan tampilin masing2 avg
def avg_dur_trips():
    conn = make_connection()
    res = get_avg_dur_trips(conn)['AvgDur']
    avg = res.values[0]
    return (f'''The average duration of bike riding is {avg:.2f} minutes per trip''')

def get_avg_dur_trips(conn):
    query = f"""SELECT AVG(duration_minutes) AS AvgDur FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

In [99]:
url = "http://127.0.0.1:5000/trips/average_duration"
res = requests.get(url, json=data_json)

In [100]:
res.text

'The average duration of bike riding is 30.59 minutes per trip'

## Create Dynamic Endpoints 

You can use the following cell to try-out your function before implementing it as an endpoint

In [ ]:
# Your Code Here
@app.route('/trips/total_duration/<bike_id>')
def total_dur_bike_id(bike_id):
    conn = make_connection()
    res = get_total_dur_bike_id(bike_id, conn)['total']
    total = res.values[0]
    return (f'''The total riding duration of bike with id {bike_id} is {total} minutes''')

def get_total_dur_bike_id(bike_id, conn):
    query = f"""SELECT SUM(duration_minutes) AS total FROM trips WHERE bikeid = {bike_id}"""
    result = pd.read_sql_query(query, conn)
    return result

In [97]:
url = "http://127.0.0.1:5000/trips/total_duration/248"
res = requests.get(url, json=data_json)

In [98]:
res.text

'The total riding duration of bike with id 248 is 66494 minutes'

## Create POST Endpoints

You can use the following cell to try-out your function before implementing it as an endpoint.

If you still find it quite difficult, here's an example case you might wanted to try on:
- input : a dictionary contained a datetime period
    - { "period" : "2015-08" }
- output:
    - Aggregation table of bike rent activities for each station in that specific period
- example code: 

```python 
input_data = request.get_json() # Get the input as dictionary
specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

# Subset the data with query 
conn = make_connection()
query = f"SELECT * FROM stations WHERE start_time LIKE ({specified_date}%)"
selected_data = pd.read_sql_query(query, conn)

# Make the aggregate
result = selected_data.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'
})

# Return the result
return result.to_json()

```

In [142]:
data = {
    "year" : "2020"
}

url = "http://127.0.0.1:5000/stations_summary"

In [143]:
res = requests.post(url, json=data)

In [144]:
res

<Response [200]>

In [145]:
res.text

'{"bikeid":{"2494":1770,"2495":1422,"2496":742,"2497":1542,"2498":1195,"2499":1852,"2501":2323,"2503":677,"2504":978,"2537":625,"2539":1951,"2540":186,"2542":1225,"2544":940,"2547":2198,"2548":1497,"2549":2205,"2552":2802,"2561":764,"2562":400,"2563":1881,"2565":381,"2566":3039,"2567":1695,"2568":278,"2569":487,"2570":639,"2571":650,"2572":1263,"2574":3068,"2575":4028,"2707":3972,"2711":1338,"2822":593,"2823":497,"3291":1031,"3292":614,"3293":514,"3294":351,"3377":2732,"3390":979,"3455":1172,"3513":1729,"3619":1486,"3621":2018,"3635":462,"3660":1385,"3684":852,"3685":829,"3686":1965,"3687":2222,"3790":777,"3791":742,"3792":1564,"3793":2571,"3794":829,"3795":1381,"3797":1105,"3798":2805,"3799":1065,"3838":1738,"3841":2110,"4047":506,"4048":1155,"4050":904,"4051":171,"4052":290,"4054":285,"4055":277,"4057":807,"4058":778,"4059":2072,"4060":1018,"4061":3481,"4062":2442,"4699":17},"duration_minutes":{"2494":63.3406779661,"2495":44.6568213783,"2496":44.9420485175,"2497":57.2522697795,"2498"

# Submission

After finishing your work of all the rubrics, the next step will be;

1. Prepare your `Bikeshare API.ipynb` file that has been edited with your code and wrangling data.
2. Prepare your `app.py` file for your Flask App. 
3. Submit your .ipynb and app.py file in your github repository. The dataset is optional to post since it has big size to post. 

